In [2]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from datetime import timedelta
import pickle

from utils import utils_scenario as utils, data_preparation as prep, data_process as proc

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 1. Load data 
 - Load forecast data
 - Load last 48 hours measurement data

In [3]:
measurement = pd.read_csv('./data/last_measurement.csv')
measurement['datetime']= pd.to_datetime(measurement['datetime'],format='%Y-%m-%d %H:%M:%S')
# measurement.head()
# forecast data
forecast = pd.read_csv('./data/last_forecast.csv')
forecast['f_date']= pd.to_datetime(forecast['f_date'],format='%Y-%m-%d %H:%M:%S')
# forecast.head()

In [4]:
#load format columns 
measurement_cols = pickle.load(open('measurement_cols.pkl', 'rb'))
forecast_cols = pickle.load(open('forecast_cols.pkl', 'rb'))
#reorder columns 
measurement = measurement[measuremengenerate_am_pm(df)t_cols]
forecast = forecast[forecast_cols]

SyntaxError: invalid syntax (<ipython-input-4-894769783ae9>, line 5)

In [ ]:
# #create pickle file of format columns
measurement_format = pd.read_csv("data/measurement_format.csv") 
forecast_format = pd.read_csv("data/forecast_format.csv") 
# measurement_cols = measurement_format.columns
# forecast_cols = forecast_format.columns
# import pickle
# with open('measurement_cols.pkl', 'wb') as f:
#     pickle.dump(measurement_cols, f)
# with open('forecast_cols.pkl', 'wb') as f:
#     pickle.dump(forecast_cols, f)



In [6]:
# # measurement = measurement.set_index('datetime')
# measurement= prep.generate_am_pm(measurement)
# measurement
# # measurement

In [7]:
measurement.shape

(329, 8)

In [8]:
#prepare raw data into same format as training df 
def prepare_x_test(measurement, forecast, past_n_steps, pred_period):
    
    #concat past n steps from measurement 
    df = measurement.set_index('datetime')
    df=proc.get_past_n_steps(df, past_n_steps)

    #calculate forecast_time
    df['forecast_time'] = df['present_time']+timedelta(hours=pred_period)

    #join forecast according to forecast time 
    forecast = forecast.set_index('f_date') 
    forecast = forecast.add_suffix('_forecast')
    df = pd.merge(df, forecast, how = 'left', left_on = 'forecast_time', right_on ='f_date')
    #add cos day
    df = proc.smooth_day_hour(df)
    cols = df.columns
    #fill missing forecasts as 0
    df.fillna(value=0, inplace=True) 
    #keep first row 
    df = df[:-past_n_steps]
    #drop timestamp columns
    df_out = df.drop(['present_time','forecast_time'], axis=1)
    return df_out, cols

test_df, cols = prepare_x_test(measurement, forecast, 48, 6 )



In [9]:
cols

Index(['speed_t-48', 'cos_wind_dir_t-48', 'sin_wind_dir_t-48', 'temp_t-48',
       'radiation_t-48', 'precip_t-48', 'season_t-48', 'speed_t-47',
       'cos_wind_dir_t-47', 'sin_wind_dir_t-47',
       ...
       'present_time', 'forecast_time', 'speed_forecast', 'temp_forecast',
       'rad_forecast', 'precip_forecast', 'cos_wind_dir_forecast',
       'sin_wind_dir_forecast', 'cos_day', 'cos_hour'],
      dtype='object', length=353)

In [10]:
pred=6
x_test = prepare_x_test(measurement, forecast, 48, pred)
x_test= np.array(x_test) #change to array 

#read 3 models for speed, cos_wind, sin_wind
xgb1= pickle.load(open('../trained_models/speed_t_'+str(pred), 'rb'))
xgb2 = pickle.load(open('../trained_models/cos_wind_dir_t_'+str(pred), 'rb'))
xgb3 = pickle.load(open('../trained_models/sin_wind_dir_t_'+str(pred), 'rb'))

#predict 
speed = xgb1.predict(x_test)
cos_wind = xgb2.predict(x_test)
sin_wind = xgb3.predict(x_test)

ValueError: ('Expecting 2 dimensional numpy.ndarray, got: ', (2,))

In [60]:
x_test.shape

(1124, 351)

In [65]:
cols2 =pickle.load(open('model_cols_past_48.pkl', 'rb'))

In [67]:
cols2.difference( cols)

Index(['am_t-0', 'am_t-1', 'am_t-10', 'am_t-11', 'am_t-12', 'am_t-13',
       'am_t-14', 'am_t-15', 'am_t-16', 'am_t-17', 'am_t-18', 'am_t-19',
       'am_t-2', 'am_t-20', 'am_t-21', 'am_t-22', 'am_t-23', 'am_t-24',
       'am_t-25', 'am_t-26', 'am_t-27', 'am_t-28', 'am_t-29', 'am_t-3',
       'am_t-30', 'am_t-31', 'am_t-32', 'am_t-33', 'am_t-34', 'am_t-35',
       'am_t-36', 'am_t-37', 'am_t-38', 'am_t-39', 'am_t-4', 'am_t-40',
       'am_t-41', 'am_t-42', 'am_t-43', 'am_t-44', 'am_t-45', 'am_t-46',
       'am_t-47', 'am_t-48', 'am_t-5', 'am_t-6', 'am_t-7', 'am_t-8', 'am_t-9'],
      dtype='object')

### Issue 1: smooth wind direction:
- the cos and sin of wind_dir seems to be wrong, re-constructed wind_dir does not agree with the real wind_dir
- the same issue could arise with the forecast processing, but we don't have the real wind_dir for forecast to check. 
- you can use the 'smooth_wind_dir' function in data_preperation.py file to correct this

In [11]:
date_to_predict = pd.to_datetime('2020-07-07 12:00:00',format='%Y-%m-%d %H:%M:%S')
# get last 48 hours measurement 
all_measurement = pd.read_csv('./data/processed_measurement.csv')
all_measurement['datetime'] = pd.to_datetime(all_measurement['datetime'],format='%Y-%m-%d %H:%M:%S')

measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                    & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
# measurement.drop(columns='wind_dir',inplace=True)
#convert cos and sin to wind_dir:
measurement['wind_dir_reconstructed'] = measurement.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)
print('reconstructed wind_dir using current cos and sin')
print(measurement[['wind_dir_reconstructed','wind_dir']].head(5))

#instead, pleas use 
measurement = prep.smooth_wind_dir(measurement)
print('using smooth_wind_dir to smooth functions')
print(measurement.head())

reconstructed wind_dir using current cos and sin
     wind_dir_reconstructed    wind_dir
107               49.145660   52.041667
108              224.833358  295.106667
109              218.301170  272.433333
110              215.627484  211.701667
111              226.018445  314.845000
using smooth_wind_dir to smooth functions
               datetime  cos_wind_dir  sin_wind_dir     speed  temp  \
107 2020-07-05 12:00:00      0.615088      0.788458  4.223333   NaN   
108 2020-07-05 13:00:00      0.424305     -0.905519  4.278333   NaN   
109 2020-07-05 14:00:00      0.042457     -0.999098  5.103333   NaN   
110 2020-07-05 15:00:00     -0.850796     -0.525496  5.020000   NaN   
111 2020-07-05 16:00:00      0.705191     -0.709017  5.300000   NaN   

      radiation  precip  season  wind_dir_reconstructed  
107  650.900000     NaN       3               49.145660  
108  748.566667     NaN       3              224.833358  
109  780.166667     NaN       3              218.301170  
110  782.7

### Issue 2: column order need to be the same as training df
- for both forecast and measurement data, the column order do not agree with the training format
- further more, the precipitation feature is abscent in forecast df, we fill with zero for now

In [12]:
measurement = measurement[measurement_format.columns]
print(measurement.head())
forecast['precip']=0
forecast = forecast[forecast_format.columns]
print(forecast.head())

NameError: name 'measurement_format' is not defined

### Issue 3: Temperature features is missing
- Temperature is an important feature
- Need to fill missing with zero

In [13]:
measurement = measurement.fillna(0)
measurement.head()

,datetime,cos_wind_dir,sin_wind_dir,speed,temp,radiation,precip,season,wind_dir_reconstructed
107,2020-07-05 12:00:00,0.615088,0.788458,4.223333,0.0,650.900000,0.0,3,49.145660
108,2020-07-05 13:00:00,0.424305,-0.905519,4.278333,0.0,748.566667,0.0,3,224.833358
109,2020-07-05 14:00:00,0.042457,-0.999098,5.103333,0.0,780.166667,0.0,3,218.301170
110,2020-07-05 15:00:00,-0.850796,-0.525496,5.020000,0.0,782.783333,0.0,3,215.627484
111,2020-07-05 16:00:00,0.705191,-0.709017,5.300000,0.0,721.450000,0.0,3,226.018445


### Now try to do prediction again

In [3]:
result  = pd.DataFrame(columns=['past_n_steps','pred_period','speed', 'cos_wind_dir','sin_wind_dir']) 
# pred_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
pred_list = [1]
past_n_steps = 48

#get forecast 
forecast = pd.read_csv('./data/last_forecast.csv')
# forecast['p_date'] = pd.to_datetime(forecast['p_date'],format='%Y-%m-%d %H:%M:%S')
forecast['f_date'] = pd.to_datetime(forecast['f_date'],format='%Y-%m-%d %H:%M:%S')
# forecast.drop(columns=['f_day','f_hour','p_date','f_period'],inplace=True)
#match orders
forecast['precip']=0
# forecast = forecast[forecast_format.columns]
forecast = forecast.fillna(0)


for i in [3,4,5]:
    # prediction date 
    date_to_predict = pd.to_datetime('2020-07-07 12:00:00',format='%Y-%m-%d %H:%M:%S') + timedelta(hours=i)
    # get last 48 hours measurement 
    all_measurement = pd.read_csv('./data/processed_measurement.csv')
    all_measurement['datetime'] = pd.to_datetime(all_measurement['datetime'],format='%Y-%m-%d %H:%M:%S')

    measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                        & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
    
    #Fix issue 1:  smooth wind direction
    measurement = prep.smooth_wind_dir(measurement) 
    #Fix issue 2:  same column order
#     measurement = measurement[measurement_format.columns]
    #Fix issue 3:  fill na
    measurement = measurement.fillna(0)

    for pred in pred_list: 
#         print(pred)
        #prepare data to be the same format as training data 
        x_test, cols = prepare_x_test(measurement, forecast, past_n_steps, pred)

        #read 3 models for speed, cos_wind, sin_wind
        xgb1= pickle.load(open('../trained_models/speed_t_'+str(pred), 'rb'))
        xgb2 = pickle.load(open('../trained_models/cos_wind_dir_t_'+str(pred), 'rb'))
        xgb3 = pickle.load(open('../trained_models/sin_wind_dir_t_'+str(pred), 'rb'))
        
        #check missing features 
        model_features = xgb1.get_booster().feature_names
        missing_features = set(model_features).difference(x_test.columns)
        if len(missing_features)>=1:
            print('some features are missing:', missing_features)
            break 
        else:
            #re-order
            x_test = x_test[model_features]
            
        #predict 
        speed = xgb1.predict(x_test)[0]
        cos_wind = xgb2.predict(x_test)[0]
        sin_wind = xgb3.predict(x_test)[0]

        #record accuracy
        result = result.append({'past_n_steps': str(past_n_steps),
                                'pred_period': str(pred),
                                'speed':round(speed,3),
                                'cos_wind_dir':cos_wind,
                                'sin_wind_dir':sin_wind}, ignore_index=True)    

    #convert cos and sin to wind_dir:
    result['wind_dir'] = result.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)
    result['datetime'] = date_to_predict + result['pred_period'].map(lambda x : timedelta(hours=int(x)))

    # get next 48hours measurement
    next_measurement = all_measurement.loc[(all_measurement.datetime>date_to_predict) \
                        & (all_measurement.datetime<=(date_to_predict + timedelta(hours=48))),]
    next_measurement['wind_dir'] = next_measurement.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)

    # merge with results
    result_vs_measurement = result[['datetime','speed','wind_dir']].merge(next_measurement[['datetime','speed','wind_dir']] \
    .rename(columns={'speed':'real_speed','wind_dir':'real_wind_dir'}))

    # append input measurement
    measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                        & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
    measurement = measurement[['datetime','speed','wind_dir']] \
    .rename(columns={'speed':'real_speed','wind_dir':'real_wind_dir'})
#     print(measurement)
print(result_vs_measurement)
#     # save file
#     (pd.concat([measurement,result_vs_measurement],axis=0).reset_index(drop=True)) \
#     .to_excel('./data/result/test2_' + str(date_to_predict)[0:13].replace(' ','_') + '.xlsx')

#     print('./data/result/test2_' + str(date_to_predict)[0:13].replace(' ','_') + '.xlsx')

NameError: name 'pd' is not defined

In [50]:
df2 = pd.DataFrame()
df['lala'] = pd.Series([1,2,3])
# df.values.flatten()
df = df.tail(2)
df = df.reset_index(drop=True)
df2['lala'] = df['lala']
df2
df

,lala
0,1
1,2


In [4]:
import pickle
pred=1
xgb1= pickle.load(open('../trained_models/cos_wind_dir_t_'+str(pred), 'rb'))
xgb1.get_booster().feature_names

['speed_t-48',
 'cos_wind_dir_t-48',
 'sin_wind_dir_t-48',
 'temp_t-48',
 'radiation_t-48',
 'precip_t-48',
 'season_t-48',
 'speed_t-47',
 'cos_wind_dir_t-47',
 'sin_wind_dir_t-47',
 'temp_t-47',
 'radiation_t-47',
 'precip_t-47',
 'season_t-47',
 'speed_t-46',
 'cos_wind_dir_t-46',
 'sin_wind_dir_t-46',
 'temp_t-46',
 'radiation_t-46',
 'precip_t-46',
 'season_t-46',
 'speed_t-45',
 'cos_wind_dir_t-45',
 'sin_wind_dir_t-45',
 'temp_t-45',
 'radiation_t-45',
 'precip_t-45',
 'season_t-45',
 'speed_t-44',
 'cos_wind_dir_t-44',
 'sin_wind_dir_t-44',
 'temp_t-44',
 'radiation_t-44',
 'precip_t-44',
 'season_t-44',
 'speed_t-43',
 'cos_wind_dir_t-43',
 'sin_wind_dir_t-43',
 'temp_t-43',
 'radiation_t-43',
 'precip_t-43',
 'season_t-43',
 'speed_t-42',
 'cos_wind_dir_t-42',
 'sin_wind_dir_t-42',
 'temp_t-42',
 'radiation_t-42',
 'precip_t-42',
 'season_t-42',
 'speed_t-41',
 'cos_wind_dir_t-41',
 'sin_wind_dir_t-41',
 'temp_t-41',
 'radiation_t-41',
 'precip_t-41',
 'season_t-41',
 'speed_